<a href="https://colab.research.google.com/github/PaulRobertH/wine-ml-portfolio/blob/main/1_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 1_data_preparation.ipynb

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import matplotlib.pyplot as plt


In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import matplotlib.pyplot as plt


Mounted at /content/drive


In [2]:
# Set your data directory
data_dir = '/content/drive/MyDrive/ML-Portfolio/data/'
file_paths = glob.glob(os.path.join(data_dir, '*.csv'))

print(f"🗂️ Found {len(file_paths)} files.")
for path in file_paths:
    print(os.path.basename(path))


🗂️ Found 22 files.
Liv-ex 100.csv
ECONOMICS_USCPI, M.csv
TVC_US10Y, 15.csv
FX_GBPUSD, 1M.csv
OANDA_EURUSD, 1M.csv
FX_IDC_GBPEUR, 1M.csv
FX_GBPJPY, 1M.csv
HKEX_DLY_HSI1!, 1M (1).csv
TVC_DXY, 1M.csv
CME_MINI_ES1!, 1M (1).csv
CME_MINI_NQ1!, 1M.csv
TVC_USOIL, 1M.csv
PEPPERSTONE_COCOA, 1M.csv
liv-ex Champagne 50.csv
liv-ex Italy 100.csv
liv-ex Rhone 100.csv
liv-ex California 50.csv
liv-ex Investables.csv
Liv-ex Fine Wine 1000.csv
Liv-ex Indices.csv
liv-ex Burgundy 150.csv
liv-ex Bordeaux Legends.csv


In [15]:
def standardise_df(path):
    import chardet
    import pandas as pd
    import numpy as np
    import os

    original_name = os.path.basename(path).replace('.csv', '')

    # Try to detect encoding
    with open(path, 'rb') as f:
        result = chardet.detect(f.read(10000))
    encoding = result['encoding']

    # Try multiple combinations
    for delimiter in [',', ';']:
        for skip in range(0, 15):
            try:
                df = pd.read_csv(path, skiprows=skip, delimiter=delimiter, encoding=encoding, engine='python')

                # Drop all-empty columns
                df = df.dropna(axis=1, how='all')
                if df.shape[1] < 2:
                    continue

                # Try to find date column
                date_cols = [col for col in df.columns if str(col).lower().strip() in ['date', 'dates', 'time']]
                if not date_cols:
                    continue

                # Parse date column
                df[date_cols[0]] = pd.to_datetime(df[date_cols[0]], errors='coerce', dayfirst=True)
                df = df.dropna(subset=[date_cols[0]])
                df = df.set_index(date_cols[0])
                df = df.sort_index()

                # Prefer 'close', 'price', 'last'
                val_col = [col for col in df.columns if str(col).lower().strip() in ['close', 'price', 'last']]
                if not val_col:
                    val_col = [col for col in df.columns if df[col].dtype in [np.float64, np.int64, np.float32]]

                if not val_col:
                    continue

                monthly = df[val_col[0]].resample('ME').last().rename(original_name)
                return monthly

            except Exception:
                continue

    raise ValueError(f"❌ Could not extract usable data from: {original_name}")


In [18]:
merged_df = pd.DataFrame()

for path in file_paths:
    try:
        col = standardise_df(path)
        merged_df = pd.concat([merged_df, col], axis=1)
    except Exception as e:
        print(f"❌ {os.path.basename(path)} skipped: {e}")

# Drop months where fewer than 5 series are present
merged_df = merged_df.dropna(thresh=5)
merged_df.tail()

# Generate table summarising each series
summary = []

for col in merged_df.columns:
    first_date = merged_df[col].first_valid_index()
    count = merged_df[col].count()
    summary.append({
        "Series": col,
        "Start Date": first_date,
        "Valid Observations": count
    })

summary_df = pd.DataFrame(summary)
summary_df = summary_df.sort_values(by="Start Date", na_position='last').reset_index(drop=True)

import IPython.display as display
display.display(summary_df)


,Series,Start Date,Valid Observations
0,Liv-ex 100,2004-01-31,202
1,"CME_MINI_NQ1!, 1M",2004-01-31,84
2,"CME_MINI_ES1!, 1M (1)",2004-01-31,90
3,"TVC_DXY, 1M",2004-01-31,102
4,"HKEX_DLY_HSI1!, 1M (1)",2004-01-31,123
5,"TVC_USOIL, 1M",2004-01-31,120
6,"FX_IDC_GBPEUR, 1M",2004-01-31,120
7,"OANDA_EURUSD, 1M",2004-01-31,72
8,"FX_GBPUSD, 1M",2004-01-31,120
9,"ECONOMICS_USCPI, M",2004-01-31,256


In [17]:
for col in merged_df.columns:
    first_date = merged_df[col].first_valid_index()
    print(f"{col}: starts at {first_date}")


Liv-ex 100: starts at 2004-01-31 00:00:00
ECONOMICS_USCPI, M: starts at 2004-01-31 00:00:00
TVC_US10Y, 15: starts at None
FX_GBPUSD, 1M: starts at 2004-01-31 00:00:00
OANDA_EURUSD, 1M: starts at 2004-01-31 00:00:00
FX_IDC_GBPEUR, 1M: starts at 2004-01-31 00:00:00
FX_GBPJPY, 1M: starts at 2004-01-31 00:00:00
HKEX_DLY_HSI1!, 1M (1): starts at 2004-01-31 00:00:00
TVC_DXY, 1M: starts at 2004-01-31 00:00:00
CME_MINI_ES1!, 1M (1): starts at 2004-01-31 00:00:00
CME_MINI_NQ1!, 1M: starts at 2004-01-31 00:00:00
TVC_USOIL, 1M: starts at 2004-01-31 00:00:00
PEPPERSTONE_COCOA, 1M: starts at 2015-01-31 00:00:00
liv-ex Champagne 50: starts at 2010-06-30 00:00:00
liv-ex Italy 100: starts at 2010-06-30 00:00:00
liv-ex Rhone 100: starts at 2010-06-30 00:00:00
liv-ex California 50: starts at 2010-06-30 00:00:00
liv-ex Investables: starts at 2010-06-30 00:00:00
Liv-ex Fine Wine 1000: starts at 2010-06-30 00:00:00
Liv-ex Indices: starts at 2010-04-30 00:00:00
liv-ex Burgundy 150: starts at 2010-06-30 00:0